In [77]:
import wand.image
from PIL import Image
from PIL.ExifTags import GPSTAGS
from PIL.ExifTags import TAGS
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import os
import cv2

In [25]:
def get_exif(filename):
    image = Image.open(filename)
    image.verify()
    return image._getexif()

def get_geotagging(exif):
    if not exif:
        raise ValueError("No EXIF metadata found")

    geotagging = {}
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx not in exif:
                raise ValueError("No EXIF geotagging found")

            for (key, val) in GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]

    return geotagging
def get_decimal_from_dms(dms, ref):

    degrees = dms[0][0] / dms[0][1]
    minutes = dms[1][0] / dms[1][1] / 60.0
    seconds = dms[2][0] / dms[2][1] / 3600.0

    if ref in ['S', 'W']:
        degrees = -degrees
        minutes = -minutes
        seconds = -seconds

    return round(degrees + minutes + seconds, 5)

def get_coordinates(geotags):
    lat = get_decimal_from_dms(geotags['GPSLatitude'], geotags['GPSLatitudeRef'])

    lon = get_decimal_from_dms(geotags['GPSLongitude'], geotags['GPSLongitudeRef'])

    return (lat,lon)



In [41]:
with wand.image.Image(filename='test.jpg') as img:
    img_resize = img.clone()
    print(img_resize.size)
    img_resize.resize(64,43)
    img_resize.format= 'jpeg'
    img_resize.save(filename='out.jpg')

(640, 428)


In [90]:
tmp1 = np.array(Image.open('test.jpg'), dtype='float')
tmp2 = cv2.imread('test.jpg')
print(tmp1.shape)

(224, 224, 3)
<class 'PIL.JpegImagePlugin.JpegImageFile'>


In [91]:
print(np.mean(tmp1[:,:,0]), np.mean(tmp1[:,:,1]), np.mean(tmp1[:,:,2]))
print(np.mean(tmp2[:,:,0]), np.mean(tmp2[:,:,1]), np.mean(tmp2[:,:,2]))

97.10319674744898 106.78013392857143 124.87531887755102
124.87531887755102 106.78013392857143 97.10319674744898


In [55]:
print(tmp1[:,:,0])

[[112. 112. 113. ...  70.  70.  72.]
 [112. 112. 113. ...  62.  64.  70.]
 [113. 113. 114. ...  57.  60.  68.]
 ...
 [ 77.  57.  62. ...  69.  53.  37.]
 [ 70.  53.  59. ...  62.  51.  45.]
 [ 59.  54.  51. ...  70.  55.  46.]]


In [78]:
def load_and_grayscale(root, path):
    # Load image.
    img = cv2.imread(root+path)
    # Convert to grayscale.
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) 
    return img

def load_and_normalize(root, path):
    # Load image.
    img = cv2.imread(root+path)
    # Convert to double and normalize.
    img = cv2.normalize(img.astype('float'), None, 
                            0.0, 1.0, cv2.NORM_MINMAX)   
    return img

In [99]:
os.path.join(os.getcwd(), 'imgs_de_augment', 'corn')

'/Users/shuilingyu/Downloads/Granular/imgs_de_augment/corn'

In [96]:
exif = get_exif('3.jpg')
geotags = get_geotagging(exif)
print(get_coordinates(geotags))

ValueError: No EXIF metadata found

In [17]:
print(img.height, img.width)

27 64


In [10]:
type(img)

wand.image.Image